In [1]:
import numpy as np 
import pandas as pd
from tqdm import tqdm

import sys
sys.path.append('../')
from retinanet import coco_eval
from retinanet import csv_eval
from retinanet import model
from retinanet import paps_eval
from retinanet import paps_train

# from retinanet import retina
from retinanet.dataloader import *
from retinanet.anchors import Anchors
from retinanet.losses import *
from retinanet.scheduler import *
from retinanet.hourglass import hg1, hg2, hg8
from retinanet.parallel import DataParallelModel, DataParallelCriterion

#Torch
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader
from torch.utils.data.sampler import SequentialSampler, RandomSampler
from torch.optim import Adam, lr_scheduler
import torch.optim as optim

# python train_paps.py --start_epoch 0 --end_epoch 120 --batch_size 24 \
#                     --saved_dir $OUT_MODEL_DIR --gpu_num 0 --num_workers 12 \
#                     --target_threshold 7 --topk 20 --filter_option 1  | 2>&1 | tee $log


In [2]:
# device = torch.device('cpu')
# device = torch.device('cuda')
GPU_NUM = 5 # 원하는 GPU 번호 입력
device = torch.device(f'cuda:{GPU_NUM}' if torch.cuda.is_available() else 'cpu')
model = hg2(device, pretrained=True, progress=False, num_classes=2)
model.to(device)

num_classes 2


HourglassNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn3): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1))
      )
    )
  )
  (layer2): Sequential(
    (0): Bottleneck(
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=Tr

In [3]:
# model

In [3]:

# train_info = np.load('../data/train.npy', allow_pickle=True, encoding='latin1').item()
# train_info

batch_size = 32
dataset_train = PapsDataset('../data/', set_name='train_2class',
                            transform=train_transforms)

train_data_loader = DataLoader(
    dataset_train,
    batch_size=batch_size,
    shuffle=True,
    num_workers=12,
#    pin_memory=True,
#    prefetch_factor=1,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=0.76s)
creating index...
index created!


In [4]:
criterion = PapsLoss(device, target_threshold=0.8, topk=10, filter_option=5)
criterion = criterion.to(device)
model.training = True

# https://gaussian37.github.io/dl-pytorch-lr_scheduler/
optimizer = optim.Adam(model.parameters(), lr = 1e-8)
scheduler = CosineAnnealingWarmUpRestarts(optimizer, T_0=20, T_mult=2, eta_max=0.0008,  T_up=5, gamma=0.5)
# CosineAnnealingWarmRestarts

In [5]:
saved_dir = '../trained_models/HourGlass/loss_smoothing' + str(0) + '/'
s_epoch= 0
e_epoch= 60

In [6]:
if os.path.isdir(saved_dir) == False :
    print('folder is made')
    os.makedirs(saved_dir)

folder is made


In [7]:
if os.path.isfile(saved_dir + 'epoch_' + str(e_epoch) +'_model.pt') :
    print('pretrainind file loading')
    state = torch.load(saved_dir + 'epoch_' + str(e_epoch) +'_model.pt')
    epoch = state['epoch']
    model.load_state_dict(state['state_dict'], strict=False)
    optimizer.load_state_dict(state['optimizer'])
    last_loss = state['loss']
else :
    last_loss = 0.6

In [8]:
paps_train.train_paps(dataloader=train_data_loader, 
                      model=model, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss)  

  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                             

0th epochs loss 0.9855369161534053 lr 1e-08 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                               

1th epochs loss 7.208486640873628 lr 0.000160008 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                

...

  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                  

59th epochs loss 0.14471552415918082 lr 0.00016000200000000002 


  0%|          | 0/465 [00:00<?, ?it/s]                                                                                                                                 

60th epochs loss 0.1477878533544079 lr 0.0002 


KeyboardInterrupt: 

In [ ]:
# state = {
#     'epoch': 0,
#     'state_dict': model.state_dict(),
#     'optimizer': optimizer.state_dict(),
#     'loss' : 0.6
# }
# torch.save(state, saved_dir + 'model.pt')

In [9]:
dataset_val = PapsDataset('../data/', set_name='val_2class',
                            transform=val_transforms)

val_data_loader = DataLoader(
    dataset_val,
    batch_size=1,
    shuffle=False,
    num_workers=4,
    collate_fn=collate_fn
)

loading annotations into memory...
Done (t=2.21s)
creating index...
index created!


In [10]:
paps_eval.evaluate_paps(dataset=dataset_val, 
  dataloader=val_data_loader, 
  model=model, 
  saved_dir=saved_dir, 
  device = device,
  threshold=0.1) 


100%|██████████| 3739/3739 [13:10<00:00,  4.73it/s] 


GT_results 17716
pred_results 53137
validation time : 791.6474401950836
Loading and preparing results...
DONE (t=0.72s)
creating index...
index created!
Loading and preparing results...
DONE (t=0.11s)
creating index...
index created!
******************total*********************
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=13.31s).
Accumulating evaluation results...
DONE (t=1.15s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.064
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.197
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.015
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.056
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.137
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.043
 Average Recall     (AR) @[ 

In [ ]:
criterion = PapsLoss(device, target_threshold=8, topk=10, filter_option=5)
criterion = criterion.to(device)
s_epoch= 60
e_epoch= 120


paps_train.train_paps(dataloader=train_data_loader, 
                      model=model, 
                      criterion=criterion,
                      saved_dir=saved_dir, 
                      optimizer=optimizer,
                      scheduler=scheduler,
                      device = device,
                      s_epoch= s_epoch,
                      e_epoch= e_epoch,
                      last_loss = last_loss)  